# Data cleaning

For this project, we are working with two main datasets: County Health Rankings (CHR - dataset 1) and County Business Patterns (CBP - dataset 2). We merge these two datasets using FIPS codes (identifier for counties) to combine a subset of health measures from dataset 1 with industry metrics from dataset 2 for all US counties.

In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import re

## 1. Read and clean main dataset 1 (County Health Rankings) 

In [3]:
# Read in County Health Rankings analytic file (dataset 1)
ds1_chr = pd.read_csv('DS1_CHR_analytic_data2021.csv', header=1)
print(f'Original shape for CHR (dataset 1): {ds1_chr.shape}')

# Convert fipscode to string, adding leading zeros as needed
ds1_chr['fipscode'] = ds1_chr['fipscode'].astype('string').str.zfill(5)

# Drop unused/duplicated columns
ds1_chr = ds1_chr.drop(columns=['statecode', 'countycode', 'year'])
# Drop non-county rows (ex. US, state-level)
ds1_chr = ds1_chr[ds1_chr['county'].str.lower().str.contains('county')]

# Exclude 'County' from name
pattern = r'(.+)(?= County)'
ds1_chr['county'] = ds1_chr['county'].str.extract(pattern)

# Keep first few columns describing location - any columns without underscore
ds1_chr_loc = ds1_chr.filter(regex=r'^(?!.*_).*$').columns.to_list()

# Manually identified subset of metrics to examine from County Health Rankings (dataset 1)
metrics_to_keep = ['v011', 'v049', 'v085', 'v004', 'v088', 'v062', 'v005', 'v023', 
                   'v125', 'v136', 'v147', 'v060', 'v139', 'v083', 'v156', 'v153', 
                   'v166', 'v051', 'v052', 'v053', 'v131', 'v144', 'v145']
metrics_to_keep = ds1_chr_loc + [s+'_rawvalue' for s in metrics_to_keep] + \
                  ['v136_other_data_'+str(n) for n in range(1,4)]
                  
# Extract columns containing _rawvalue and _other_data, and columns describing location
ds1_chr_metrics = ds1_chr.filter(items=metrics_to_keep) 

del(ds1_chr_loc, metrics_to_keep)
print(f'Final shape for CHR (dataset 1): {ds1_chr_metrics.shape}')
ds1_chr_metrics.head()

Original shape for CHR (dataset 1): (3194, 690)
Final shape for CHR (dataset 1): (3007, 29)


,v011_rawvalue,v049_rawvalue,v085_rawvalue,v004_rawvalue,v088_rawvalue,v062_rawvalue,v005_rawvalue,v023_rawvalue,v125_rawvalue,v136_rawvalue,...,v053_rawvalue,v131_rawvalue,v144_rawvalue,v145_rawvalue,v136_other_data_1,v136_other_data_2,v136_other_data_3,fipscode,state,county
2,0.330,0.144739,0.100473,0.000468,0.000340,0.000286,6650.0,0.027281,10.4,0.136310,...,0.159731,0.000412,0.132530,0.158092,0.120326,0.011161,0.015436,01001,AL,Autauga
3,0.300,0.187370,0.131690,0.000702,0.000493,0.000986,3471.0,0.027258,7.2,0.129967,...,0.209780,0.000645,0.116116,0.147556,0.119875,0.010179,0.003349,01003,AL,Baldwin
4,0.412,0.124670,0.135040,0.000322,0.000365,0.000122,5314.0,0.037952,9.4,0.137106,...,0.196913,0.000567,0.181321,0.189547,0.125943,0.019587,0.006529,01005,AL,Barbour
5,0.374,0.154206,0.105991,0.000536,0.000223,0.000268,6690.0,0.030628,10.0,0.092552,...,0.166696,0.001206,0.150659,0.171418,0.082637,0.005640,0.005640,01007,AL,Bibb
6,0.330,0.158333,0.141160,0.000207,0.000190,0.000173,4440.0,0.026687,10.6,0.101257,...,0.187009,0.000242,0.148125,0.171296,0.074689,0.018125,0.011600,01009,AL,Blount


## 2. Read main dataset 2 (County Business Patterns) and add industry categories

In [5]:
# Read in county business patterns (CBP) dataset (dataset 2)
ds2_cbp = pd.read_csv('DS2_CBP_cbp21co.txt', sep=',', 
                      dtype = {'fipstate': 'str', 'fipscty': 'str'})

print(f'Original shape for CBP (dataset 2): {ds2_cbp.shape}')

# Combine state and county FIPS codes to get 5-digit code, for merging with other datasets
ds2_cbp['fipscode'] = ds2_cbp['fipstate'] + ds2_cbp['fipscty']
# Drop state and county FIPS codes - will only use 5-digit FIPS code (fipscode)
ds2_cbp.drop(columns=['fipstate', 'fipscty'], inplace=True)
# Move fipscode to the front
column_to_move = ds2_cbp.pop('fipscode') # Removes specified column
ds2_cbp.insert(0, 'fipscode', column_to_move) # Reinsert column to specified index

del(column_to_move) # Deleted to save space
print(f'Original shape for CBP (dataset 2): {ds2_cbp.shape}')
ds2_cbp.head()

Original shape for CBP (dataset 2): (1090164, 23)
Original shape for CBP (dataset 2): (1090164, 22)


,fipscode,naics,emp_nf,emp,qp1_nf,qp1,ap_nf,ap,est,n<5,...,n20_49,n50_99,n100_249,n250_499,n500_999,n1000,n1000_1,n1000_2,n1000_3,n1000_4
0,01001,------,G,11216,G,97944,G,424826,924,442,...,100,31,8,N,N,N,N,N,N,N
1,01001,11----,G,78,G,1325,G,5107,10,9,...,N,N,N,N,N,N,N,N,N,N
2,01001,113///,G,68,G,1213,G,4563,7,6,...,N,N,N,N,N,N,N,N,N,N
3,01001,1133//,G,68,G,1213,G,4563,7,6,...,N,N,N,N,N,N,N,N,N,N
4,01001,11331/,G,68,G,1213,G,4563,7,6,...,N,N,N,N,N,N,N,N,N,N


## 3. Read dataset 3 (NAICS) with industry categories

Although several levels of industry categories are included, we chose to keep the highest level categories, to limit the number of industries we have to compare between counties.

In [6]:
# Read in NAICS to Industry file (dataset 3) - contains industry descriptions
ds3_naics_to_ind = pd.read_csv('DS3_naics2017.txt', sep=',', encoding='latin1') 
print(f'Original shape for NAICS (dataset 3): {ds3_naics_to_ind.shape}')
# Set column names to lowercase to match CBP (dataset 2), for merging
ds3_naics_to_ind.columns = ds3_naics_to_ind.columns.str.lower()
# Rename 'description' to something more descriptive
ds3_naics_to_ind = ds3_naics_to_ind.rename(columns={'description': 'Industry Description'})

# Filter NAICS data - only keep highest level codes (2 digits followed by 4 hyphens)
pattern = r'([0-9]+)(?=----)'
ds3_naics_to_ind = ds3_naics_to_ind[ds3_naics_to_ind['naics'].str.match(pattern)]

print(f'Final shape for NAICS (dataset 3): {ds3_naics_to_ind.shape}')
ds3_naics_to_ind.head()

Original shape for NAICS (dataset 3): (2003, 2)
Final shape for NAICS (dataset 3): (20, 2)


,naics,Industry Description
1,11----,"Agriculture, Forestry, Fishing and Hunting"
36,21----,"Mining, Quarrying, and Oil and Gas Extraction"
84,22----,Utilities
109,23----,Construction
182,31----,Manufacturing


## 4. Merge all datasets

In [7]:
# Merge dataset 3 (NAICS to Industry) with dataset 2 (County Business Patterns) on NAICS codes
merged_ds2and3_cbp_ind = pd.merge(ds3_naics_to_ind, ds2_cbp, on='naics')

# Merge with merged CHR (dataset 1) on FIPS codes
merged_ds1_2_chr_cbr_allcounties = pd.merge(merged_ds2and3_cbp_ind, 
                                            ds1_chr_metrics, on = 'fipscode')

# Rearrange columns, so identifying columns are in front (FIPS, state, county)
colnames_to_move = ['fipscode', 'state', 'county']
cols_to_move = merged_ds1_2_chr_cbr_allcounties[colnames_to_move]
merged_ds1_2_chr_cbr_allcounties.drop(columns=colnames_to_move, inplace=True)
merged_ds1_2_chr_cbr_allcounties = pd.merge(cols_to_move, 
                                            merged_ds1_2_chr_cbr_allcounties, 
                                            left_index=True, right_index=True)

del(colnames_to_move, cols_to_move)

# Save dataframe in CSV for use in following notebooks
merged_ds1_2_chr_cbr_allcounties.to_csv('_merged_ds1_2_chr_cbr_allcounties.csv')

print(f'Final shape of merged dataset: {merged_ds1_2_chr_cbr_allcounties.shape}')
merged_ds1_2_chr_cbr_allcounties.head()

Final shape of merged dataset: (47560, 51)


,fipscode,state,county,naics,Industry Description,emp_nf,emp,qp1_nf,qp1,ap_nf,...,v166_rawvalue,v051_rawvalue,v052_rawvalue,v053_rawvalue,v131_rawvalue,v144_rawvalue,v145_rawvalue,v136_other_data_1,v136_other_data_2,v136_other_data_3
0,01001,AL,Autauga,11----,"Agriculture, Forestry, Fishing and Hunting",G,78,G,1325,G,...,0.805674,55869,0.232061,0.159731,0.000412,0.13253,0.158092,0.120326,0.011161,0.015436
1,01001,AL,Autauga,21----,"Mining, Quarrying, and Oil and Gas Extraction",G,87,G,1224,G,...,0.805674,55869,0.232061,0.159731,0.000412,0.13253,0.158092,0.120326,0.011161,0.015436
2,01001,AL,Autauga,22----,Utilities,G,138,H,4508,G,...,0.805674,55869,0.232061,0.159731,0.000412,0.13253,0.158092,0.120326,0.011161,0.015436
3,01001,AL,Autauga,23----,Construction,G,507,G,5364,G,...,0.805674,55869,0.232061,0.159731,0.000412,0.13253,0.158092,0.120326,0.011161,0.015436
4,01001,AL,Autauga,31----,Manufacturing,G,1086,H,19475,H,...,0.805674,55869,0.232061,0.159731,0.000412,0.13253,0.158092,0.120326,0.011161,0.015436


<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=935c12df-95fc-47b5-a033-2935b9193bd3' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>